In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,075 kB]
Get:13 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-24 22:03:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.61MB/s    in 0.6s    

2022-07-24 22:03:24 (1.61 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17747349|R2EI7QLPK4LF7U|B00U7LCE6A|     106182406|CCleaner Free [Do...|Digital_Software|          4|            0|          0|   N|                Y|          Four Stars|      So far so good| 2015-08-31|
|         US|   10956619|R1W5OMFK1Q3I3O|B00HRJMOM4|     162269768|ResumeMaker Profe...|Digital_Software|          3|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [7]:
# Create the customers_table DataFrame
customers_df = df.groupby('customer_id').agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46639223|             1|
|   50407297|             1|
|   52707015|             1|
|   21817779|             1|
|     178254|             1|
|    1442229|             1|
|   12811624|             1|
|   40840841|             1|
|   48319420|             1|
|   16205135|             1|
|   12115907|             1|
|   30190304|             1|
|   43013424|             1|
|   51495952|             1|
|   16558777|             1|
|   53013109|             2|
|   50065664|             1|
|    3534576|             1|
|   36456998|             1|
|   31609314|             1|
+-----------+--------------+
only showing top 20 rows



In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates(['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007QV9EXU|Mavis Beacon Keyb...|
|B00905OPAM|Quicken Legal Bus...|
|B0091KPDL6|AVG Internet Secu...|
|B00EKRUMNO|BUSINESS CARD STU...|
|B00G05RTS0|Microsoft Windows...|
|B00H9MX1G2|eMedia Piano For ...|
|B00JPRV0PI|WordPerfect Offic...|
|B00K7FX3PM|Home Designer Arc...|
|B00M9GTMQ6|Intuit Quicken St...|
|B00P9GBX7O|Proactive Parenta...|
|B00SZ1J8L8|TechSmith Snagit ...|
|B00VGR1AZA|Adobe Acrobat Pro...|
|B00VIPANZ8|Home Designer Sui...|
|B001M4JFTU|Quicken Starter E...|
|B007SOLPQ4|Math Coloring Boo...|
|B007USG5JK|Adobe CS6 Design ...|
|B008UVL8RK|Audials Moviebox ...|
|B009EF74S8|iDevice Magic Pla...|
|B009G6SNKS|Adobe Acrobat XI Pro|
|B009H6LFIY|        Rostta Stone|
+----------+--------------------+
only showing top 20 rows



In [9]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2EI7QLPK4LF7U|   17747349|B00U7LCE6A|     106182406| 2015-08-31|
|R1W5OMFK1Q3I3O|   10956619|B00HRJMOM4|     162269768| 2015-08-31|
| RPZWSYWRP92GI|   13132245|B00P31G9PQ|     831433899| 2015-08-31|
|R2WQWM04XHD9US|   35717248|B00FGDEPDY|     991059534| 2015-08-31|
|R1WSPK2RA2PDEF|   17710652|B00FZ0FK0U|     574904556| 2015-08-31|
|R11JVGRZRHTDAS|   42392705|B004KPKSRQ|     306022575| 2015-08-31|
|R2B8468OKXXYE2|   52845868|B00B1TFNTW|      54873662| 2015-08-31|
|R2HGGCCZSSNUCB|   15696503|B00M9GTJLY|     103182180| 2015-08-31|
| REEE4LHSVPRV9|    9723928|B00H9A60O4|     608720080| 2015-08-31|
|R25OMUUILFFHI9|   23522877|B008XAXAC4|      87969525| 2015-08-31|
|R2966PB8UBD5BM|   17022093|B00MHZ6Z64|     249773946| 2015-08-31|
|R1OU91L2G5H6H1|   11635690|B00OPCQ70Q|     956532818| 2015-08

In [10]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2EI7QLPK4LF7U|          4|            0|          0|   N|                Y|
|R1W5OMFK1Q3I3O|          3|            0|          0|   N|                Y|
| RPZWSYWRP92GI|          1|            1|          2|   N|                Y|
|R2WQWM04XHD9US|          5|            0|          0|   N|                Y|
|R1WSPK2RA2PDEF|          4|            1|          2|   N|                Y|
|R11JVGRZRHTDAS|          5|            4|          4|   N|                N|
|R2B8468OKXXYE2|          1|            1|          1|   N|                N|
|R2HGGCCZSSNUCB|          1|            0|          0|   N|                Y|
| REEE4LHSVPRV9|          1|            0|          0|   N|                Y|
|R25OMUUILFFHI9|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cjek06tehkv0.us-east-1.rds.amazonaws.com:5432/dataviz"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [14]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)